# VeloVAE (fullvb) benchmark on dyngen data

Notebook benchmarks velocity and latent time inference using VeloVAE (fullvb) on dyngen-generated data.

## Library imports

In [1]:
import velovae as vv

import numpy as np
import pandas as pd
import torch

import anndata as ad
import scvelo as scv

from rgv_tools import DATA_DIR
from rgv_tools.benchmarking import get_velocity_correlation

2024-12-14 01:30:12.923192: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-14 01:30:12.938701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734136212.957171 3032484 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734136212.962640 3032484 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 01:30:12.983527: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## General settings

In [2]:
scv.settings.verbosity = 3

## Constants

In [3]:
torch.manual_seed(0)
np.random.seed(0)

In [4]:
DATASET = "dyngen"

In [5]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)
    (DATA_DIR / DATASET / "processed" / "velovae_vae").mkdir(parents=True, exist_ok=True)

## Velocity pipeline

In [ ]:
velocity_correlation = []

for filename in (DATA_DIR / DATASET / "processed").iterdir():
    if filename.suffix != ".zarr":
        continue

    adata = ad.io.read_zarr(filename)

    try:
        rate_prior = {"alpha": (0.0, 1.0), "beta": (0.0, 0.5), "gamma": (0.0, 0.5)}
        vae = vv.VAE(adata, tmax=20, dim_z=5, device="cuda:0", full_vb=True, rate_prior=rate_prior)
        config = {}
        vae.train(adata, config=config, plot=False, embed="pca")

        # Output velocity to adata object
        vae.save_anndata(adata, "fullvb", DATA_DIR / DATASET / "processed" / "velovae_vae", file_name="velovae.h5ad")

        adata.layers["velocity"] = adata.layers["fullvb_velocity"].copy()

        velocity_correlation.append(
            get_velocity_correlation(
                ground_truth=adata.layers["true_velocity"], estimated=adata.layers["velocity"], aggregation=np.mean
            )
        )

    except Exception as e:  # noqa: BLE001
        # Append np.nan in case of an error and optionally log the error
        print(f"An error occurred: {e}")
        velocity_correlation.append(np.nan)

Estimating ODE parameters...


  0%|          | 0/105 [00:00<?, ?it/s]

Detected 59 velocity genes.
Estimating the variance...


  0%|          | 0/105 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


...

  0%|          | 0/83 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/83 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.56, 0.24297477479231852), (0.44, 0.7779664767114342)
KS-test result: [0. 1. 1.]
Initial induction: 60, repression: 23/83
Learning Rate based on Data Sparsity: 0.0000
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 2, test iteration: 2
*********       Stage 1: Early Stop Triggered at epoch 9.       *********
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


  0%|          | 0/300 [00:00<?, ?it/s]

## Data saving

In [11]:
if SAVE_DATA:
    pd.DataFrame({"velocity": velocity_correlation}).to_parquet(
        path=DATA_DIR / DATASET / "results" / "velovae_fullvb_correlation.parquet"
    )